Data from: **The adaptive value of probability distortion and risk-seeking in macaques’ decision-making**
A. Nioche, N. P. Rougier, M. Deffains, S. Bourgeois-Gironde, S. Ballesta and T. Boraud
https://doi.org/10.1098/rstb.2019.0668

## Convert Excel file to CSV if necessary

In [ ]:
# Check if csv file has been created
# !!! Parsing the excel file takes a few minutes
import pandas as pd
import os.path

data_filename = "./data/RSTB.csv"
if not os.path.exists(data_filename):
    original_data = pd.read_excel("./data/RSTB.xlsx")
    original_data.to_csv(data_filename)
else:
    original_data = pd.read_csv(data_filename)

## Make data compatible with new format

Process the original dataset to ensure that tasks are named properly.
The original dataset is untouched and the processed dataset is saved using an alternative filename.

| Name           | Type     | Signification                 |
| :------------- | :------- | :---------------------------- |
| **subject_id** | string   | Identification of the subject |
| **date**       | datetime | Date whe then trial was made   | 
| **task_id**    | integer  | Identification of the task    | 
| **P_left**     | float    | Reward probability of the left stimulus |
| **V_left**     | float    | Reward amount of the left stimulus |
| **P_right**    | float    | Reward probability of the right stimulus |
| **P_right**    | float    | Reward amount of the right stimulus |
| **response**   | int      | Response (0: left, 1: right) |
| **reward**     | int      | Reward delivered (1) or not (0) |
| **RT**         | int      | Response time (ms) |


## Filter, rename & retype fields

In [ ]:
# Keep only relevant fields
data = original_data[['monkey',
                      'date',
                      'stim_left_p',
                      'stim_left_x0',
                      'stim_right_p',
                      'stim_right_x0',
                      'choice',
                      'stim_dice_output',
                      'time_response']].copy()
data["Type"] = "0"

# Fix a spurious "None" at the end of date 
data["date"]= data["date"].astype("S10").astype(str)

# Rename fields
data = data.rename(columns={"monkey" :           "subject_id",
                            "date" :             "date",
                            "Type" :             "task_id",
                            "stim_left_p" :      "P_left",
                            "stim_left_x0" :     "V_left",
                            "stim_right_p" :     "P_right",
                            "stim_right_x0" :    "V_right",
                            "choice" :           "response",
                            "stim_dice_output" : "reward",
                            "time_response" :    "RT"})

# Convert task_id type (from string to int)
data["task_id"] = 0
data["task_id"] = pd.to_numeric(data["task_id"])

# Convert date type (from string to datetime64)
data["date"] = pd.to_datetime(data["date"])

## Assign task id

In [ ]:
# We assign task ids based on probablities and values
p1, x1 = data["P_left"], data["V_left"]
p2, x2 = data["P_right"], data["V_right"]

data.loc[(p1 == p2) & (x1 <  0) & (x2 >   0), "task_id"] = 1
data.loc[(p1 == p2) & (x2 <  0) & (x1 >   0), "task_id"] = 1

data.loc[(p1 == p2) & (x1 >  0) & (x2 >   0), "task_id"] = 2

data.loc[(p1 == p2) & (x1 <  0) & (x2 <   0), "task_id"] = 3

data.loc[(p1 >  p2) & (x1 >  0) & (x1 == x2), "task_id"] = 4
data.loc[(p2 >  p1) & (x1 >  0) & (x1 == x2), "task_id"] = 4

data.loc[(p1 >  p2) & (x1 <  0) & (x1 == x2), "task_id"] = 5
data.loc[(p2 >  p1) & (x1 <  0) & (x1 == x2), "task_id"] = 5

data.loc[(p1 <  p2) & (x1 > x2) & (x2 >   0), "task_id"] = 6
data.loc[(p2 <  p1) & (x2 > x1) & (x1 >   0), "task_id"] = 6

data.loc[(p1 <  p2) & (x1 < x2) & (x2 <   0), "task_id"] = 7
data.loc[(p2 <  p1) & (x2 < x1) & (x1 <   0), "task_id"] = 7

In [ ]:
import os
filename, extension = os.path.splitext(data_filename)
filename = f"{filename}-processed{extension}"

print("Saving new dataset... ", end="")
data.to_csv(filename)
print("done!")
print("New dataset:", filename)